In [1]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


In [2]:
# Set the URL for the CUNY academic calendar
url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
